# Query Example

In [1]:
import os, time
from influxdb_client_3 import InfluxDBClient3, Point

token = ""
org = 'Jay-IOx'
host = 'eu-central-1-1.aws.cloud2.influxdata.com'

client = InfluxDBClient3(host=host, token=token, org=org)

In [2]:
database="python"

data = {
  "point1": {
    "location": "Klamath",
    "species": "bees",
    "count": 23,
  },
  "point2": {
    "location": "Portland",
    "species": "ants",
    "count": 30,
  },
  "point3": {
    "location": "Klamath",
    "species": "bees",
    "count": 28,
  },
  "point4": {
    "location": "Portland",
    "species": "ants",
    "count": 32,
  },
  "point5": {
    "location": "Klamath",
    "species": "bees",
    "count": 29,
  },
  "point6": {
    "location": "Portland",
    "species": "ants",
    "count": 40,
  },
}

for key in data:
  point = (
    Point("census")
    .tag("location", data[key]["location"])
    .field(data[key]["species"], data[key]["count"])
  )
  client.write(database=database, record=point)
  time.sleep(1) # separate points by 1 second

print("Complete. Return to the InfluxDB UI.")
    


Complete. Return to the InfluxDB UI.


In [4]:
query = """SELECT *
FROM census
WHERE time >= now() - interval '24 hours'
AND ('bees' IS NOT NULL OR 'ants' IS NOT NULL)"""

# Execute the query
table = client.query(query=query, database=database, language='sql')

# Convert to dataframe
df = table.to_pandas().sort_values(by="time")
print(df)

   ants  bees  location                          time
0   NaN  23.0   Klamath 2023-08-09 10:13:46.302280197
3  30.0   NaN  Portland 2023-08-09 10:13:47.468192006
1   NaN  28.0   Klamath 2023-08-09 10:13:48.619137933
4  32.0   NaN  Portland 2023-08-09 10:13:49.688372628
2   NaN  29.0   Klamath 2023-08-09 10:13:50.819573138
5  40.0   NaN  Portland 2023-08-09 10:13:51.893667576


In [17]:
query = """SELECT mean(ants) as ant_mean, mean(bees) as bee_mean
FROM census
WHERE time >= now() - 24h"""

# Execute the query
table = client.query(query=query, database=database, language='influxql')

# Convert to dataframe
df2 = table.to_pandas()
print(df2)

  iox::measurement       time  ant_mean   bee_mean
0           census 1970-01-01      34.0  26.666667


In [35]:


p = point = Point.measurement("mem").tag("location", "europe").field("level", 1)

print(p)

mem,location=europe level=1i
